In [1]:
import argparse
import logging
import os
import random
import pandas as pd
from dataclasses import dataclass
from itertools import chain
from typing import Optional, Union
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import datasets
import torch
from datasets import load_dataset, load_metric, DatasetDict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    default_data_collator,
    DataCollatorForSeq2Seq,
    AdamW,
    SchedulerType,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from promptsource.templates import DatasetTemplates
pd.set_option("display.max_rows", 1200)


/opt/conda/envs/tz/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
n = 1000
model_dir = "output_1000_plus"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy[i-1] = temp['accuracy'].iloc[0]
# model_dir = "output"
# df2 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy2 = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy2[i-1] = temp['accuracy'].iloc[0]
# # model_dir = "output2"
# # df3 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# # accuracy3 = np.zeros(n-1)
# # for i in range(1, n):
# #     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
# #     accuracy3[i-1] = temp['accuracy'].iloc[0]
    

In [ ]:
df.to_csv(f'/home/gikok/{model_dir}/stats.csv')

In [ ]:
n = 342
model_dir = "output_no_GA"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats.csv')
even = np.arange(0, len(df)-1, 2)
even1 = np.arange(1, len(df), 2)
df1 = df.iloc[even]
df2 = df.iloc[even1]

In [ ]:
e = df.groupby(df.index//200).median()
et =  df.groupby(df.index//200).max()
eb =  df.groupby(df.index//200).quantile(0.25)
e2 = df2.groupby(df2.index//200).max()
et2 =  df2.groupby(df2.index//200).quantile(0.75)
eb2 =  df2.groupby(df2.index//200).quantile(0.25)
l = np.arange(len(e))
plt.rcParams['font.size'] = 16
# plt.plot(l, e["training_loss"], label = 'Task 1', c='green')
plt.plot(l, et["test_loss"], c='green', ls='--')
# plt.plot(l, eb["training_loss"], c='green', ls='--')

plt.plot(l, e2["test_loss"], label = 'Task 2', c='red', alpha=0.6)
# plt.plot(l, et2["training_loss"], c='red', ls='--')
# plt.plot(l, eb2["training_loss"], c='red', ls='--')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("2 options")
#plt.ylim(3e-1, 10)
plt.legend()
# plt.yscale('log')
plt.show()
plt.plot(l, abs(e["lm_grad_pct"]), label = 'Task 1')
plt.plot(l, abs(e2["lm_grad_pct"]), label = 'Task 2')
plt.xlabel('Epoch')
plt.yscale('log')
plt.title("Output Vocabulary")
plt.ylabel('<Gradient>')
plt.legend()
#plt.yscale('log')
plt.show()
plt.plot(l, abs(e["em_grad_pct"]), label= 'Task 1')
plt.plot(l, abs(e2["em_grad_pct"]), label='Task 2')
plt.xlabel('Epoch')
plt.ylabel('<Gradient>')
plt.title("Input Vocabulary")
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["lm_delta_pct"], '.', label = 'LM delta (prompt 1)')
plt.plot(l, e2["lm_delta_pct"], '.', label = 'LM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-0.2, 0.2])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["em_delta_pct"], '.', label= 'EM delta (prompt 1)')
plt.plot(l, e2["em_delta_pct"], '.', label=' EM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-2e-3, 2e-3])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
torch.cuda.device_count()

In [2]:
#Load model and tokenizer

model_name = "bigscience/T0_3B"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded")

model.to('cuda:0')
print("Moved model to GPUs")
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer2 = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded")

model2.to('cuda:0')
print("Moved model to GPUs")



Model and tokenizer loaded
Moved model to GPUs
Model and tokenizer loaded
Moved model to GPUs


In [44]:
import itertools
o = ['is red', 'vechicle', 'is Nissan']
li = list(itertools.permutations(o))
lis = [f"{', '.join(v)}" for v in li]
b = '; '.join(lis)

In [76]:
inp = f"list: car, dog, cat. in the previous list, how many items that are similar to animals?   1, 2, 3, 4, 5, 6, 7, 8, 9, 10?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs#.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Q: list: car, dog, cat. in the previous list, how many items that are similar to animals?   1, 2, 3, 4, 5, 6, 7, 8, 9, 10?
A: 10


In [43]:
tokenizer.encode('Nissan')

[19026, 1]

In [ ]:
inp = "What is the top speed of a leopard?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=False))

In [ ]:
embeddings = list(model.named_parameters())[0][1]
lm_head = list(model.named_parameters())[-1][1]
n_new = 100
og = embeddings[:-n_new,:].detach().cpu()
new = embeddings[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("Embedding Layer")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("New EM")
plt.show()
n_new = 100
og = lm_head[:-n_new,:].detach().cpu()
new = lm_head[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
#plt.ylim(0, 15)
plt.title("Language Model Head")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.ylim(0, 10)
plt.title("New LM")
plt.show()

In [ ]:
df = pd.read_parquet('data/miniprompts005.parquet.gzip')
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
w = 1123
print(df['input'].iloc[w][:-12] + ", " + ', '.join(items))
print(df['target'].iloc[w])
print(items.remove(items[0]))


In [6]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    BeamSearchScorer,
)
# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(100, eos_token_id=model.config.eos_token_id),
    ]
)

In [16]:
w = 1123
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
#items.remove('30493350')
#inp = "query: 'artificial potted plant'. for the above query what item_no is the best match?"
inp = "what is 40503607?"
inp1 = inp  + ', '.join(items)
#inp = "99429408 and 50401335 are dining tables. is the previous sentence correct? yes or no?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)#, logits_processor=logits_processor)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
#print("A: " + ', '.join((dict.fromkeys((tokenizer.decode(outputs[0], skip_special_tokens=True).split(','))))))

Q: what is 40503607?
A: The narrator is describing the events of the day.


In [15]:
w = 1123
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
#items.remove('30493350')
#inp = "query: 'artificial potted plant'. for the above query what item_no is the best match?"
inp = "what is 40503607?"
inp1 = inp  + ', '.join(items)
#inp = "99429408 and 50401335 are dining tables. is the previous sentence correct? yes or no?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model2.generate(inputs)#, logits_processor=logits_processor)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
#print("A: " + ', '.join((dict.fromkeys((tokenizer.decode(outputs[0], skip_special_tokens=True).split(','))))))

Q: what is 40503607?
A: The narrator is a narrator.


In [ ]:
names = pd.read_parquet("data/item_names_100.parquet.gzip").values.tolist()
for i, item in enumerate(names):
    inp = f"describe what you know about {item[0]}"
    inputs = tokenizer.encode(
        inp, return_tensors='pt')
    inputs = inputs.to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(inputs)
    print(item[1])
    print("Q: " + inp)
    print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("")
    print("")
    print("**************************************************************")
    print("")

In [14]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos2 = torch.nn.functional.cosine_similarity

In [37]:
v.shape

torch.Size([32100, 2048])

In [44]:
r = cos2(v, lm[-93], dim=-1)
[tokenizer.decode(torch.tensor(r).topk(10, largest=True)[1], skip_special_tokens=False)]

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


['duplicate Yes Brown yes Pra Sage herbal woodland 2.233']

In [22]:
items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
names = pd.read_parquet("data/seven_names.parquet.gzip")["name"].values.tolist()

l = ["furniture", "couch", "sofa", "chair", "shelf", "kitchen", "table", "bed"]
inputs = tokenizer.batch_encode_plus(
    l, return_tensors="pt", padding=True
).to('cuda:0')
inds = torch.tensor([inputs['input_ids'][i].max() for i in range(len(l))])
cos = torch.nn.functional.cosine_similarity
emb = list(model.named_parameters())[0][1]
lm = list(model.named_parameters())[-1][1]
emb_old_tokens = emb[:-100]
lm_old_tokens = lm[:-100]

#tt = emb[inds]
#tt = lm[inds]

emb_new_tokens = emb[-100:]
lm_new_tokens = lm[-100:]

emb_top10 = []
lm_top10 = []
for i in range(len(emb_new_tokens)):
    new_emb_token = emb_new_tokens[i]
    new_lm_token = lm_new_tokens[i]

    emb_top10_similar = torch.tensor(cos(new_emb_token, emb_old_tokens, dim=-1)).topk(10, largest=True)
    lm_top10_similar = torch.tensor(cos(new_lm_token, lm_old_tokens, dim=-1)).topk(10, largest=True)

    emb_top10 += [[tokenizer.decode(emb_top10_similar[1], skip_special_tokens=False)]]
    lm_top10 += [[tokenizer.decode(lm_top10_similar[1], skip_special_tokens=False)]]
df = pd.DataFrame({'item_no':items,'name':names, 'top_10_emb':emb_top10, 'top_10_lm':lm_top10})
df

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


,item_no,name,top_10_emb,top_10_lm
0,40278675,foerbaettra leg off white,[legs leg Legktion feet cabinets cabinet seleg...,[legs leg feet Leg cabinetsktion cabinetlegthi...
1,40503607,indo agra floral rug handmade grey dark grey,[rugrugs Rug carpet rug wool Carpet large Wool],[rugrugs carpet Rug large Carpetrug wool a]
2,70461583,lerhyttan door light gray,[door doors cabinet Door cabinets Cabinet Door...,[door doorst cabinetdoors Door cabinetsyh]
3,40494491,baestis hook beige,[hook Hook dog wall hang hooked dogs Dog Hang ...,[hook wall Hook hang doghook hooked hanging do...
4,50460952,aurdal shelf dark gray,[shelf Shelf shelves auelf shelvingd closetalr],[shelfd shelves Shelfr aualral shelving closet]
5,40527397,sandbrodd throw beige,[blanket throw Throw throwingthrownhrew Blanks...,[blanket throws Throw throwingdthrown Blanka ]
6,29303964,bror cabinet unit white,[storage cabinet Storage cabinets Cabinet cupb...,[storage cabinet cabinets Storage cupboard Cab...
7,19485605,enhet kitchen white gray frame white frame,[kitchen Kitchen cabinet sink cabinets Cabinet...,[kitchen cabinet cabinets sink Kitchen cupboar...
8,60371762,hemnes bookcase black brown,[hem book books shelves shelfnescase Shelfelf ],[hem booknes bookshbookbookscase shelves shelf]
9,39398933,besta tv bench,[vt ab Ve standrV],[tvas e-di.]


In [9]:
queries = pd.read_parquet("data/item_namequeries100.parquet.gzip")
df = pd.DataFrame({'item_no':items,'top_10_emb':emb_top10, 'top_10_lm':lm_top10})
df = df.merge(queries, how='inner', on='item_no')

In [21]:
df2 = df.copy()

In [11]:
pd.concat(df.iloc[0], df2.iloc[0])

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [23]:
d = pd.DataFrame()
c = 0
c1 = 0
c2 = 0
for i in range(300):
    if c == 0:
        d = d.append(df.iloc[c1], ignore_index=True)
        c1 += 1
    if c == 1:
        d = d.append(df2.iloc[c2], ignore_index=True)
        c2 += 1
    if c == 2:
        d = d.append({'item_no':" ", 'name':" ", 'top_10_emb':" ", 'top_10_lm':" "}, ignore_index=True)
        c = -1
    c+=1

In [24]:
d

,item_no,name,top_10_emb,top_10_lm
0,40278675,foerbaettra leg off white,[legs leg Legktion feet cabinets cabinet seleg...,[legs leg feet Leg cabinetsktion cabinetlegthi...
1,40278675,foerbaettra leg off white,[distracted Familien sanguinfrant psychologica...,[lingual alăturslippinghrenfaxrisen Nantesreic...
2,,,,
3,40503607,indo agra floral rug handmade grey dark grey,[rugrugs Rug carpet rug wool Carpet large Wool],[rugrugs carpet Rug large Carpetrug wool a]
4,40503607,indo agra floral rug handmade grey dark grey,[Komfortoricarfrucht schönfuzNeill possédschüt...,[Semester swelling ressourcesisé Paypal Rezept...
5,,,,
6,70461583,lerhyttan door light gray,[door doors cabinet Door cabinets Cabinet Door...,[door doorst cabinetdoors Door cabinetsyh]
7,70461583,lerhyttan door light gray,[détailléhistorischen peinture logiciel santé ...,[Lebens folosesc Karlsruheaider stärker abge W...
8,,,,
9,40494491,baestis hook beige,[hook Hook dog wall hang hooked dogs Dog Hang ...,[hook wall Hook hang doghook hooked hanging do...


In [27]:
d.to_excel('new_vs_old_seed.xlsx')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
def resample(model, layer, n_new, init_type='random'):

    new_tensor = list(model.named_parameters())[layer][1].detach().cpu()
    l = [f'rank_{i}' for i in range(1,21)]

    if init_type=='random':
        for i in range(len(new_tensor[-1, :])):
            val, bin = np.histogram(new_tensor[:-n_new, i], 10000)
            pdf = val / sum(val)
            cdf = np.cumsum(pdf)
            b = (bin[1:] + bin[:-1]) / 2
            new_tensor[-n_new:, i] = torch.tensor(
                np.interp(np.random.random(n_new), cdf, b)
            )
    if init_type=='targeted':
        search_names = pd.read_parquet('data/item_no_to_query.parquet.gzip')
        for i in range(n_new):
            ranks = search_names[search_names['item_no']==items[i]][l]
            ranks = ranks[ranks!='PLACEHOLDER'].dropna(axis=1)
            rank_tokens = [tokenizer.encode(r)[:-1] for r in ranks.values.tolist()[0]]
            all_tokens = list(chain.from_iterable(rank_tokens))
            tensor_list = [new_tensor[t,:]*1 for t in all_tokens]
            temp_tensor = torch.stack(tensor_list)
            new_tensor[-n_new+i,:] = torch.mean(temp_tensor, axis=0)
    data = list(model.named_parameters())[layer][1].data
    data[:, :] = new_tensor

items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
tokenizer.add_tokens(items)
model.resize_token_embeddings(len(tokenizer))
model2.resize_token_embeddings(len(tokenizer))


Embedding(32200, 2048)

In [4]:
from itertools import chain
list(chain.from_iterable(rank_tokens))

In [93]:
torch.cat(temp_t, temp_t2).shape

tensor([-11.2589,  -5.6888,  -2.3916,  ...,  11.4940,  19.7456,   4.5235],
       device='cuda:0', grad_fn=<SelectBackward>)

In [59]:
len(all_tokens)

50